ETL with Spark (Local)

In [ ]:
from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, TimestampType

# import pyspark.sql.functions as F

In [ ]:
import pandas as pd
import glob

In [ ]:
p = glob.glob("data/*.json")

In [ ]:
p

In [ ]:
pd.read_json(p)

In [ ]:
data = "github_events_01.json"

In [ ]:
data_2 = "github_events_02.json"

In [ ]:
spark = SparkSession.builder \
    .appName("ETL") \
    .getOrCreate()

In [ ]:
data_folder = "data"

In [ ]:
data = spark.read.option("multiline", "true").json(data_folder)

In [ ]:
# data = spark.read.option("multiline", "true").json(data_2)

In [ ]:
data.show()

In [ ]:
data.printSchema()

In [ ]:
data.select("id", "type").show()

In [ ]:
data.createOrReplaceTempView("staging_events")

In [ ]:
table = spark.sql("""
    select
        *
        
    from
        staging_events
""").show()

In [ ]:
table = spark.sql("""
    select
        id
        , type
        , created_at
        , to_date(created_at) as date
        , year(created_at) as year
        , actor.login
        , actor.url as actor_url
        , repo.name
        , repo.url as repo_url
        
    from
        staging_events
""")

In [ ]:
table.show()

In [ ]:
output_csv = "output_csv"
output_parquet = "output_parquet"

In [ ]:
table.write.partitionBy("year").mode("overwrite").csv(output_csv)

In [ ]:
table.write.partitionBy("year").mode("overwrite").parquet(output_parquet)


In [ ]:
table = spark.sql("""
    select
        id
        , type
        , created_at
        , day(created_at) as day
        , month(created_at) as month
        , year(created_at) as year
        , date(created_at) as date
    from
        staging_events
""")

In [ ]:
table.show()

In [ ]:
destination = "events"

In [ ]:
table.write.partitionBy("year", "month", "day").mode("overwrite").csv(destination)


In [ ]:
table.write.partitionBy("date").mode("overwrite").csv(destination)


In [ ]:
table = spark.sql("""
    select
        actor.login
        , id as event_id
        , actor.url as actor_url
    from
        staging_events
""")
destination = "actors"
table.write.mode("overwrite").csv(destination)

In [ ]:
table = spark.sql("""
    select
        repo.name
        , id as event_id
        , repo.url as repo_url
        
    from
        staging_events
""")
destination = "repos"
table.write.mode("overwrite").csv(destination)